In [46]:
!cd container; ./build_and_push.sh drug-prediction

WARNING! Using --password via the CLI is insecure. Use --password-stdin.
WARNING! Your password will be stored unencrypted in /home/ec2-user/.docker/config.json.
Configure a credential helper to remove this warning. See
https://docs.docker.com/engine/reference/commandline/login/#credentials-store

Login Succeeded
Sending build context to Docker daemon  58.88kB
Step 1/12 : ARG REGION=us-west-2
Step 2/12 : FROM continuumio/miniconda3
 ---> 406f2b43ea59
Step 3/12 : RUN apt-get update
 ---> Using cache
 ---> eecc3f1de6b3
Step 4/12 : RUN apt-get install -y build-essential
 ---> Using cache
 ---> 5863123421aa
Step 5/12 : RUN conda install -y -c conda-forge rdkit
 ---> Using cache
 ---> c397a3a2c5b4
Step 6/12 : RUN pip install sagemaker-containers
 ---> Using cache
 ---> 118bf6576a55
Step 7/12 : RUN pip install git+https://github.com/samoturk/mol2vec
 ---> Using cache
 ---> b5f7dd89dd32
Step 8/12 : RUN pip install scikit-learn==0.21.3
 ---> Using cache
 ---> d186ea7db994
Step 9/12 : ENV PATH=

In [40]:
from sagemaker import get_execution_role

role = get_execution_role()

In [41]:
import os
import subprocess

instance_type = 'local'

if subprocess.call('nvidia-smi') == 0:
    ## Set type to GPU if one is present
    instance_type = 'local_gpu'
    
print("Instance type = " + instance_type)

Instance type = local


In [42]:
from sagemaker.estimator import Estimator

hyperparameters = {'dev-mode': True}

estimator = Estimator(role=role,
                      train_instance_count=1,
                      train_instance_type=instance_type,
                      image_name='drug-prediction:latest',
                      hyperparameters=hyperparameters)

estimator.fit('file:///home/ec2-user/SageMaker/baseline/data')

Creating tmpi7nqgliu_algo-1-qvagh_1 ... 
Attaching to tmpi7nqgliu_algo-1-qvagh_12mdone
algo-1-qvagh_1  | 2020-01-20 22:35:01,264 sagemaker-containers INFO     No GPUs detected (normal if no gpus installed)
algo-1-qvagh_1  | 2020-01-20 22:35:01,265 sagemaker-containers INFO     Failed to parse hyperparameter dev-mode value True to Json.
algo-1-qvagh_1  | Returning the value itself
algo-1-qvagh_1  | 2020-01-20 22:35:01,280 sagemaker-containers INFO     Failed to parse hyperparameter dev-mode value True to Json.
algo-1-qvagh_1  | Returning the value itself
algo-1-qvagh_1  | 2020-01-20 22:35:01,283 sagemaker-containers INFO     No GPUs detected (normal if no gpus installed)
algo-1-qvagh_1  | 2020-01-20 22:35:01,296 sagemaker-containers INFO     Failed to parse hyperparameter dev-mode value True to Json.
algo-1-qvagh_1  | Returning the value itself
algo-1-qvagh_1  | 2020-01-20 22:35:01,298 sagemaker-containers INFO     No GPUs detected (normal if no gpus installed)
algo-1-qvagh_1  | 2020-01

# Run A Training Job

In [47]:
import sagemaker as sage

sess = sage.Session()

In [48]:
import boto3

client = boto3.client('sts')
account = client.get_caller_identity()['Account']

my_session = boto3.session.Session()
region = my_session.region_name

algorithm_name = 'drug-prediction'

ecr_image = '{}.dkr.ecr.{}.amazonaws.com/{}:latest'.format(account, region, algorithm_name)

print(ecr_image)

405787280505.dkr.ecr.us-west-2.amazonaws.com/drug-prediction:latest


In [49]:
from sagemaker.estimator import Estimator

data_location = 's3://jdurago-insight-2020a/data'
output_path = 's3://jdurago-insight-2020a/output/baseline/'
max_run_time = 3*60*60 # train for max of 3 hours
hyperparameters = {'dev-mode': False}

# instance_type = 'ml.p2.xlarge'
instance_type = 'ml.m5.4xlarge'

estimator = Estimator(role=role,
                      train_instance_count=1,
                      train_instance_type=instance_type,
                      image_name=ecr_image,
                      hyperparameters=hyperparameters,
                     output_path = output_path,
                     train_max_run=max_run_time)

estimator.fit(data_location)


2020-01-20 23:01:13 Starting - Starting the training job...
2020-01-20 23:01:15 Starting - Launching requested ML instances......
2020-01-20 23:02:17 Starting - Preparing the instances for training...
2020-01-20 23:03:00 Downloading - Downloading input data
2020-01-20 23:03:00 Training - Downloading the training image.....2020-01-20 23:03:46,055 sagemaker-containers INFO     No GPUs detected (normal if no gpus installed)
2020-01-20 23:03:46,056 sagemaker-containers INFO     Failed to parse hyperparameter dev-mode value False to Json.
Returning the value itself
2020-01-20 23:03:47,517 sagemaker-containers INFO     Failed to parse hyperparameter dev-mode value False to Json.
Returning the value itself
2020-01-20 23:03:47,520 sagemaker-containers INFO     No GPUs detected (normal if no gpus installed)
2020-01-20 23:03:47,530 sagemaker-containers INFO     Failed to parse hyperparameter dev-mode value False to Json.
Returning the value itself
2020-01-20 23:03:47,532 sagemaker-containers INF